In [39]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [40]:
env = GridworldEnv()

In [41]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    V_prev = np.zeros(env.nS)
    i = 1
    def printGrid(v):
        for i, j in enumerate(v):
            if i>0 and i % int(np.sqrt(env.nS)) == 0:
                print "\n\t%.1f" % j,
            else:
                print "\t%.1f" % j,
        print
                
    while True:
        delta = 0
        # for all states
        for s, vs in  enumerate(V):
            # calc value based on current reward plus next step reward following policy
            v_curr = v_next = 0.0
            for a, p in enumerate(policy[s]):
                for prob, next_state, reward, done in env.P[s][a]:
                    v_curr += p * prob * reward
                    v_next += p * prob * V_prev[next_state]
            V[s] = v_curr + discount_factor * v_next
            delta = max(delta, np.abs(V[s] - vs))
        V_prev = np.copy(V)
        if i < 6:
            print "Iteration #%d" % i
            printGrid(V)
            i +=1
        if delta < theta:
            break
    return np.array(V)


In [42]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

Iteration #0
	0.0 	-1.0 	-1.0 	-1.0 
	-1.0 	-1.0 	-1.0 	-1.0 
	-1.0 	-1.0 	-1.0 	-1.0 
	-1.0 	-1.0 	-1.0 	0.0
Iteration #1
	0.0 	-1.8 	-2.0 	-2.0 
	-1.8 	-2.0 	-2.0 	-2.0 
	-2.0 	-2.0 	-2.0 	-1.8 
	-2.0 	-2.0 	-1.8 	0.0
Iteration #2
	0.0 	-2.4 	-2.9 	-3.0 
	-2.4 	-2.9 	-3.0 	-2.9 
	-2.9 	-3.0 	-2.9 	-2.4 
	-3.0 	-2.9 	-2.4 	0.0
Iteration #3
	0.0 	-3.1 	-3.8 	-4.0 
	-3.1 	-3.7 	-3.9 	-3.8 
	-3.8 	-3.9 	-3.7 	-3.1 
	-4.0 	-3.8 	-3.1 	0.0
Iteration #4
	0.0 	-3.7 	-4.7 	-4.9 
	-3.7 	-4.5 	-4.8 	-4.7 
	-4.7 	-4.8 	-4.5 	-3.7 
	-4.9 	-4.7 	-3.7 	0.0


In [15]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)